# Geoquoteerde eindoefening

## Max Valkenburg & Emile Knaepen

## 1. (S)Arima model

In [1]:
#imports
import numpy as np
import pandas as pd

from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot

In [2]:
#read data
SolarData = pd.read_excel('SolarForecast.xls')

In [3]:
SolarData.head()

,DateTime,Most recent forecast [MW],Day-Ahead forecast [MW],Week-Ahead forecast [MW],Real-time Upscaled Measurement [MW],Corrected Upscaled Measurement [MW],Monitored Capacity [MWp],Day-Ahead forecast (11h00) [MW]
0,01/11/2020 00:00,0.0,0.0,0.0,0.0,0.0,4037.14,0.0
1,01/11/2020 00:15,0.0,0.0,0.0,0.0,0.0,4037.14,0.0
2,01/11/2020 00:30,0.0,0.0,0.0,0.0,0.0,4037.14,0.0
3,01/11/2020 00:45,0.0,0.0,0.0,0.0,0.0,4037.14,0.0
4,01/11/2020 01:00,0.0,0.0,0.0,0.0,0.0,4037.14,0.0


In [4]:
SolarData.describe()

,Most recent forecast [MW],Day-Ahead forecast [MW],Week-Ahead forecast [MW],Real-time Upscaled Measurement [MW],Corrected Upscaled Measurement [MW],Monitored Capacity [MWp],Day-Ahead forecast (11h00) [MW]
count,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.00,2880.000000
mean,182.063976,184.772465,172.949524,181.968361,174.375819,4037.14,186.468549
std,341.131936,347.318288,308.062869,349.078161,345.762503,0.00,347.514519
min,0.000000,0.000000,0.000000,0.000000,0.000000,4037.14,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,4037.14,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,4037.14,0.000000
75%,227.157500,220.230000,262.690000,220.555000,187.170000,4037.14,230.605000
max,1580.210000,1606.320000,1420.990000,1800.640000,1748.420000,4037.14,1575.510000


In [5]:
SolarData.info()
np.asarray(SolarData)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   DateTime                             2880 non-null   object 
 1   Most recent forecast [MW]            2880 non-null   float64
 2   Day-Ahead forecast [MW]              2880 non-null   float64
 3   Week-Ahead forecast [MW]             2880 non-null   float64
 4   Real-time Upscaled Measurement [MW]  2880 non-null   float64
 5   Corrected Upscaled Measurement [MW]  2880 non-null   float64
 6   Monitored Capacity [MWp]             2880 non-null   float64
 7   Day-Ahead forecast (11h00) [MW]      2880 non-null   float64
dtypes: float64(7), object(1)
memory usage: 180.1+ KB


array([['01/11/2020 00:00', 0.0, 0.0, ..., 0.0, 4037.14, 0.0],
       ['01/11/2020 00:15', 0.0, 0.0, ..., 0.0, 4037.14, 0.0],
       ['01/11/2020 00:30', 0.0, 0.0, ..., 0.0, 4037.14, 0.0],
       ...,
       ['30/11/2020 23:15', 0.0, 0.0, ..., 0.0, 4037.14, 0.0],
       ['30/11/2020 23:30', 0.0, 0.0, ..., 0.0, 4037.14, 0.0],
       ['30/11/2020 23:45', 0.0, 0.0, ..., 0.0, 4037.14, 0.0]],
      dtype=object)

In [6]:
#rename column zodat we die makkelijk kunnnen kiezen
SolarData = SolarData.rename({"Corrected Upscaled Measurement [MW]" : "Measurement"}, axis = 'columns')

In [7]:
#datetime column van string naar datetime omzetten
#01/11/2020 12:00

SolarData.DateTime = [datetime.strptime(x, '%d/%m/%Y %H:%M') for x in SolarData.DateTime]
SolarData.DateTime = pd.DatetimeIndex(SolarData.DateTime).to_period('15Min')
Solar = pd.Series(SolarData.Measurement, index = SolarData.DateTime)

In [8]:
#arima model maken
model = ARIMA(Solar, order=(5,1,0))

In [9]:
SolarData_fit = model.fit()

C:\Users\maxPC\anaconda3\envs\SmartSystems\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\maxPC\anaconda3\envs\SmartSystems\lib\site-packages\numpy\core\fromnumeric.py:3583: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\maxPC\anaconda3\envs\SmartSystems\lib\site-packages\numpy\core\_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\maxPC\anaconda3\envs\SmartSystems\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\maxPC\anaconda3\envs\SmartSystems\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1030: RuntimeWa

LinAlgError: Schur decomposition solver error.